**Gradient Boosting**

XGboost lgbm

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


In [0]:
from sklearn import preprocessing
import xgboost as xgb

In [0]:
#Train/Test Split
train_transaction = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [0]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
adversarial_features = ["TransactionDT"]
train = train.drop(adversarial_features, axis = 1)
test = test.drop(adversarial_features, axis = 1)
print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

(60953, 432)
(61686, 431)


In [0]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [0]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,68.500,4,13926,-999.0,150.0,2,142.0,2,315.0,87.0,19.0,-999.0,0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-999.0,13.0,-999.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,13.0,13.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987001,29.000,4,2755,404.0,150.0,3,102.0,2,325.0,87.0,-999.0,-999.0,17,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,0.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987002,59.000,4,4663,490.0,150.0,4,166.0,3,330.0,87.0,287.0,-999.0,36,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,0.0,315.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987003,50.000,4,18132,567.0,150.0,3,117.0,3,476.0,87.0,-999.0,-999.0,54,0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,-999.0,-999.0,-999.000000,-999.000000,84.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987004,50.000,1,4497,514.0,150.0,3,102.0,2,420.0,87.0,-999.0,-999.0,17,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,-999.0,-999.0,-999.0,...,0.0,70787.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,100.0,2,-999.0,-480.0,2,2,166.0,-999.0,542.0,144.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,2,2,8,116,32.0,104,4,2,1,2,2,2,500
2987005,49.000,4,5937,555.0,150.0,4,226.0,3,272.0,87.0,36.0,-999.0,17,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,0.0,0.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987006,159.000,4,12308,360.0,150.0,4,166.0,3,126.0,87.0,0.0,-999.0,54,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,0.0,0.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987007,422.500,4,12695,490.0,150.0,4,226.0,3,325.0,87.0,-999.0,-999.0,30,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.000000,-999.000000,0.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,0,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,-999.0,-999.0,-999.0,0,0,0,0,0,-999.0,0,0,0,0,0,0,0,0
2987008,15.000,1,2803,100.0,150.0,4,226.0,3,337.0,87.0,-999.0,-999.0,2,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .1)

In [0]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(54857, 431)
(6096, 431)
(54857,)
(6096,)


In [0]:
eval_set = [(X_val, y_val)]

In [0]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=6,
                        learning_rate=0.1,
                        subsample=0.7,
                        colsample_bytree=0.7,
                        missing=-999,
                       )

clf.fit(X_train, y_train, eval_set = eval_set, early_stopping_rounds = 10)

[0]	validation_0-error:0.021982
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.021161
[2]	validation_0-error:0.020997
[3]	validation_0-error:0.020833
[4]	validation_0-error:0.020505
[5]	validation_0-error:0.020505
[6]	validation_0-error:0.020505
[7]	validation_0-error:0.020341
[8]	validation_0-error:0.020177
[9]	validation_0-error:0.020177
[10]	validation_0-error:0.020013
[11]	validation_0-error:0.020669
[12]	validation_0-error:0.020669
[13]	validation_0-error:0.020505
[14]	validation_0-error:0.020341
[15]	validation_0-error:0.020013
[16]	validation_0-error:0.019521
[17]	validation_0-error:0.019685
[18]	validation_0-error:0.019521
[19]	validation_0-error:0.019849
[20]	validation_0-error:0.019685
[21]	validation_0-error:0.019685
[22]	validation_0-error:0.019357
[23]	validation_0-error:0.019357
[24]	validation_0-error:0.019521
[25]	validation_0-error:0.019029
[26]	validation_0-error:0.019193
[27]	validation_0-error:0.018701
[28]	validation_0-er

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.7, verbosity=1)

In [0]:
from sklearn.metrics import roc_auc_score


In [0]:
xgb_pred_val = clf.predict_proba(X_val)[:,1]

In [0]:
print(roc_auc_score(y_val, xgb_pred_val))

0.9183298505348162


In [0]:
import lightgbm as lgb

In [0]:
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        X_train[f] = X_train[f].astype("category")
        X_train[f] = X_train[f].astype("category")
        X_test[f] = X_test[f].astype("category")  

In [0]:
params = {'application': 'xentropy',
          'boosting': 'gbdt',
          'learning_rate': 0.1,
          'bagging_fraction': 0.6,
          'feature_fraction': 0.6,
          'verbosity': -1,
          'data_random_seed': 24,
          'early_stop': 10,
          'verbose_eval': 50,
          'num_rounds': 10000}

In [0]:
d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_val, label = y_val)
watchlist = [d_train, d_valid]
num_rounds = params.pop('num_rounds')
verbose_eval = params.pop('verbose_eval')
early_stop = None
if params.get('early_stop'):
    early_stop = params.pop('early_stop')
model = lgb.train(params,
                  train_set=d_train,
                  num_boost_round=num_rounds,
                  valid_sets=watchlist,
                  verbose_eval=verbose_eval,
                  early_stopping_rounds=early_stop)

Training until validation scores don't improve for 10 rounds.
[50]	training's xentropy: 0.045352	valid_1's xentropy: 0.0652869
[100]	training's xentropy: 0.0316952	valid_1's xentropy: 0.0584751
[150]	training's xentropy: 0.0232723	valid_1's xentropy: 0.0552374
[200]	training's xentropy: 0.0177387	valid_1's xentropy: 0.0535817
[250]	training's xentropy: 0.0134238	valid_1's xentropy: 0.0521509
Early stopping, best iteration is:
[265]	training's xentropy: 0.0124657	valid_1's xentropy: 0.051696


In [0]:
lgb_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
print(roc_auc_score(y_val, lgb_pred_val))

0.9443788960119005


In [0]:
lgb_pred_val

array([0.00034043, 0.00076682, 0.00337507, ..., 0.00228053, 0.00801945,
       0.00581657])

In [0]:
print(roc_auc_score(y_val, (xgb_pred_val + lgb_pred_val)/2))

0.939424762697457


In [0]:
xgb_pred = clf.predict_proba(X_test)[:,1]
lgb_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [0]:
((xgb_pred_val + lgb_pred_val)/2).mean()

0.021868371709701583